# 네이버 쇼핑리뷰 감성분석
- GRU
- Mecab 형태소 분석기

## mecab 설치

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh


## 데이터 탐색

In [3]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt'
df=pd.read_table(url,names=['ratings','reviews'])
df.head()

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [5]:
#평정 4,5점인 데이터 = 1(긍정), 나머지=0(부정)
df['label']=df.ratings.apply(lambda x: 1 if x >= 4 else 0)
df.head()

,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [6]:
df.label.value_counts() 
#비율 확인. 비슷하다.

0    100037
1     99963
Name: label, dtype: int64

In [7]:
df.shape

(200000, 3)

### 데이터 전처리

In [8]:
df.isna().sum().sum()

0

In [9]:
#중복데이터 확인
df.reviews.nunique()

199908

In [10]:
#중복데이터 제거
df.drop_duplicates(subset=['reviews'],inplace=True)
df.shape

(199908, 3)

### 한글 이외 데이터 제거

In [11]:
df.reviews = df.reviews.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
#df.reviews = df.reviews.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','') 
#힣 다음에 스페이스 넣고, ''스페이스 안 넣고, str strip 안 넣음

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [12]:
#null 데이터 생기면 제거. 근데 없는 듯.
df.reviews.replace('',np.nan,inplace=True)
df.isna().sum().sum()

0

- 훈련/테스트 데이터셋 분리

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.reviews.values, df.label.values, stratify=df.label.values,
    test_size=0.2, random_state=2022
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((159926,), (39982,), (159926,), (39982,))

## 토큰화

In [14]:
from konlpy.tag import Mecab
mecab = Mecab()

In [15]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']


In [16]:
from tqdm.notebook import tqdm

train_data = []
for sentence in tqdm(X_train):
    morphs = mecab.morphs(sentence)
    tmp_X = [word for word in morphs if word not in stopwords]
    #케라스 토크나이저는 문장으로 안 주고 숫자로 주기 때문에 ' '.join 안 해줘도 됨.
    train_data.append(tmp_X)

  0%|          | 0/159926 [00:00<?, ?it/s]

In [29]:
from tqdm.notebook import tqdm

test_data = []
for sentence in tqdm(X_test):
    morphs = mecab.morphs(sentence)
    tmp_X = [word for word in morphs if word not in stopwords]
    #케라스 토크나이저는 문장으로 안 주고 숫자로 주기 때문에 ' '.join 안 해줘도 됨.
    test_data.append(tmp_X)

  0%|          | 0/39982 [00:00<?, ?it/s]

- 인코딩


In [36]:
import numpy as np
import tensorflow as tf
seed=2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer
t=Tokenizer()
t.fit_on_texts(train_data)

In [38]:
len(t.word_index)

40704

In [39]:
threshold=3
total_cnt=len(t.word_index)
rare_cnt=0 #등장 빈도가 threshold 보다 작은 단어의 갯수
total_freq=0 #훈련 데이터의 전체 단어의 빈도수의 합
rare_freq=0 #등장 빈도가 threshold 보다 작은 단어의 등장 빈도수의 합


In [40]:
for key, value in t.word_counts.items():
    total_freq +=value
    if value <threshold:
        rare_cnt +=1
        rare_freq +=value
    

In [41]:
print('단어 집합(vocabulary)의 크기 :', total_cnt)
print(f'등장 빈도가 {threshold - 1}번 이하인 희귀 단어의 수: {rare_cnt}')
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)


단어 집합(vocabulary)의 크기 : 40704
등장 빈도가 2번 이하인 희귀 단어의 수: 23294
단어 집합에서 희귀 단어의 비율: 57.22779088050315
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.0769616318809065


- 등장 빈도가 2회 이하인 단어는 제외하고 토큰화

In [42]:
#0번은 패팅 토큰, 1번은 out of value(oov) 토큰을 채울거임. 이걸 고려해 +2를 함.
vocab_size=total_cnt - rare_cnt + 2
vocab_size

17412

In [43]:
t = Tokenizer(num_words=vocab_size, oov_token='OOV')
t.fit_on_texts(train_data)
X_train=t.texts_to_sequences(train_data)
X_test=t.texts_to_sequences(test_data)

In [44]:
max(len(s) for s in X_train),sum(map(len,X_train)) / len(X_train)

(86, 16.367832622587947)

In [46]:
#리뷰 길이를 60으로 설정 (패딩 기준)
max_len=60

In [47]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

## GRU 모델
- 임베딩 벡터 갯수 = 100
- GRU 유닛 갯수 = 128

In [48]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [50]:
model=Sequential([
                  Embedding(vocab_size, 100, input_length=max_len),
                  GRU(128),
                  Dense(1,activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 100)           1741200   
                                                                 
 gru (GRU)                   (None, 128)               88320     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,829,649
Trainable params: 1,829,649
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.compile('adam','binary_crossentropy',['accuracy'])

In [52]:
model_path='best-navershopping-lstm.h5'
mc=ModelCheckpoint(model_path,verbose=1,save_best_only=True)
es=EarlyStopping(patience=3)

In [53]:
hist=model.fit(
    X_train,y_train,validation_split=0.2,
    epochs=30,batch_size=128,callbacks=[mc,es]
)

Epoch 1/30
1000/1000 [==============================] - ETA: 0s - loss: 0.2831 - accuracy: 0.8905
Epoch 1: val_loss improved from inf to 0.23784, saving model to best-navershopping-lstm.h5
1000/1000 [==============================] - 18s 10ms/step - loss: 0.2831 - accuracy: 0.8905 - val_loss: 0.2378 - val_accuracy: 0.9125
Epoch 2/30
 995/1000 [============================>.] - ETA: 0s - loss: 0.2073 - accuracy: 0.9263
Epoch 2: val_loss improved from 0.23784 to 0.22559, saving model to best-navershopping-lstm.h5
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2073 - accuracy: 0.9263 - val_loss: 0.2256 - val_accuracy: 0.9179
Epoch 3/30
 998/1000 [============================>.] - ETA: 0s - loss: 0.1753 - accuracy: 0.9390
Epoch 3: val_loss did not improve from 0.22559
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1753 - accuracy: 0.9391 - val_loss: 0.2407 - val_accuracy: 0.9137
Epoch 4/30
 995/1000 [============================>.] - ETA: 0s - loss: 

In [55]:
best_model=load_model(model_path)
best_model.evaluate(X_test,y_test)

1250/1250 [==============================] - 5s 4ms/step - loss: 0.2316 - accuracy: 0.9145


[0.2315959930419922, 0.914511501789093]

In [54]:
def sentiment_predict(review, tokenizer=t, max_len=max_len):
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',review).strip()
    morphs = mecab.morphs(review)
    morphs = [word for word in morphs if word not in stopwords]
    encoded = tokenizer.texts_to_sequences([morphs])
    padded = pad_sequences(encoded, maxlen=max_len)
    score = float(best_model.predict(padded))
    return f'긍정({score*100:.2f}%)' if score > 0.5 else f'부정({(1-score)*100:.2f}%)'


In [59]:
sentiment_predict('리뷰쓰기도 귀찮아')

'부정(95.98%)'